In [1]:
import pandas as pd
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import gutenberg, stopwords

from book_functions import *

# Reloads above files as they are saved internally
%reload_ext autoreload
%autoreload 2

# Future Considerations

In [2]:
df = pd.read_csv('updating_df.csv')

In [3]:
len(df)

21926

In [3]:
# with open('goodreads_updated.json') as f:
df = pd.read_json('goodreads_updated.json', orient='index').reset_index(drop=True)

## NLP Topic Modeling

Preprocessing Data

In [17]:
processed_docs = df['description'].map(preprocess)
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=20, no_above=0.66, keep_n=10000)

Bag of Words on dataset

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_doc_2001 = bow_corpus[2001]

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=4, workers=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
processed_docs[7090447] # example to view

In [ ]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))
    
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

Then on unseen document

## Topic Clustering

Pearson Correlation from https://github.com/nikitaa30/Content-based-Recommender-System/blob/master/recommender_system.py

In [32]:
def bracket_match(text):
    check = []
    need = 0
    for i in text:
        if i == "(":
            check.append('open')
        elif i == ")":
            check.append('close')
    for idx in range(len(check)):
        if len(check) == 0:
            break
        else:
            if check[idx] == 'open':
                print(check)
                if 'close' in check[idx:]:
                    del check[idx]
                    del check[check[idx:].index('close') + idx]

    need += len(check)
    return need

In [33]:

bracket_match("(())")

['open', 'open', 'close', 'close']


IndexError: list index out of range